<a href="https://www.kaggle.com/code/lazuardialmuzaki/hybrid-algorithm-ga-ts-for-hub-allocation-problem?scriptVersionId=143803091" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# pd.options.display.float_format = '{:,}'.format

## Load all datasets
Comprises of 5 different dataset that encompasses different number of node. 
First data includes 10 nodes, then 25, 55, 81, and 100 respectively.
All of each dataset may consist different matrix of demand flow and matrix of cost.


In [ ]:
flow = pd.read_excel('/kaggle/input/nodes-datasets/CAB10.xlsx', header=None, sheet_name='flow_matrix')
cost = pd.read_excel('/kaggle/input/nodes-datasets/CAB10.xlsx', header=None, sheet_name='cost_matrix')

flow_matrix = np.array(flow)
cost_matrix = np.array(cost)

flow = pd.read_excel('/kaggle/input/nodes-datasets/CAB25.xlsx', header=None, sheet_name='flow_matrix')
cost = pd.read_excel('/kaggle/input/nodes-datasets/CAB25.xlsx', header=None, sheet_name='cost_matrix')

flow_matrix_25 = np.array(flow)
cost_matrix_25 = np.array(cost)

flow = pd.read_excel('/kaggle/input/nodes-datasets/TR55.xlsx', header=None, sheet_name='flow_matrix')
cost = pd.read_excel('/kaggle/input/nodes-datasets/TR55.xlsx', header=None, sheet_name='cost_matrix')

flow_matrix_55 = np.array(flow)
cost_matrix_55 = np.array(cost)

flow = pd.read_excel('/kaggle/input/nodes-datasets/TR81.xlsx', header=None, sheet_name='flow_matrix')
cost = pd.read_excel('/kaggle/input/nodes-datasets/TR81.xlsx', header=None, sheet_name='cost_matrix')

flow_matrix_81 = np.array(flow)
cost_matrix_81 = np.array(cost)

flow = pd.read_excel('/kaggle/input/nodes-datasets/RGP100.xlsx', header=None, sheet_name='flow_matrix')
cost = pd.read_excel('/kaggle/input/nodes-datasets/RGP100.xlsx', header=None, sheet_name='cost_matrix')

flow_matrix_100 = np.array(flow)
cost_matrix_100 = np.array(cost)

In [ ]:
flow_matrix

In [ ]:
cost_matrix

## Network Cost


Interhub flow is entitled to discounted rates, defined as following formula:

In [ ]:
def flow_loc(interhub_flow):
    flow_discounted=0
    if interhub_flow < 50000:
        alpha = 1
        intercept = 0
    elif interhub_flow < 100000:
        alpha = 0.8
        intercept = 10000
    elif interhub_flow < 200000:
        alpha = 0.6
        intercept = 30000
    elif interhub_flow >= 200000:
        alpha = 0.4
        intercept = 70000
    else:
        raise ValueError('Invalid interhub flow')
    return alpha, intercept

For flows which are not interhub (for example non-hub node to hub, and hub to non-hub node), the cost is defined as follows:

In [ ]:
def distribution_collection_cost(solution,flow_matrix,cost_matrix):
    total_distribution_cost=0
    for i in range(len(solution)):
        for j in range(len(solution)):
            cost=flow_matrix[i][j]*(cost_matrix[i][solution[i]]+cost_matrix[solution[j]][j])
            total_distribution_cost+=cost
    return total_distribution_cost

As accumulated interhub flow and its corresponding discount has been calculated previously, below code shows the calculation for its cost:

In [ ]:
def inter_hub_cost(solution,flow_matrix,cost_matrix):
    inter_hub_cost=0
    hub_indices =set(solution)
    for hub1 in hub_indices:
        for hub2 in hub_indices:
            if hub1!=hub2:
                inter_hub_flow=0
                for i in range(len(solution)):
                    for j in range(len(solution)):                   
                      
                        if solution[i]==hub1 and solution[j]==hub2:
                            inter_hub_flow+=flow_matrix[i][j]
                alpha, intercept = flow_loc(inter_hub_flow)
                inter_hub_cost+=cost_matrix[hub1][hub2]*(alpha*inter_hub_flow + intercept)
                
    return inter_hub_cost


Total cost of interhub and non interhub (distribution and collection)

In [ ]:
def total_cost(solution,flow_matrix,cost_matrix):
    total_cost = distribution_collection_cost(solution,flow_matrix,cost_matrix) + inter_hub_cost(solution,flow_matrix,cost_matrix)
    return total_cost
    

**Checking Developed Code**

The code is tested using the optimal solution of node array in code below, the result shows that the optimal value is found, denoting that the code for computing hub cost is correct

In [ ]:
total_cost([3,8,8,3,3,8,6,3,8,6], flow_matrix, cost_matrix)

### Setting up the genetic alrogithm part

## Initial Solution

#### Flowbased
Determining which nodes selected as hub is based on node to node flow magnitude

In [ ]:
def weighted_flowbased_initial_solution_GA(flow_matrix, n_hub, n_node, population):
    
    assign_array_set = []
    hub_array_set = []
    
    total_flow = []
    node_index = []
    two_thirds_biggest_flow_index = []

    
    for node in range(n_node):
        flow = 0
        
        for other_node in range(n_node):
            if node == other_node:
                continue
            
            flow += flow_matrix[node][other_node]
            flow += flow_matrix[other_node][node]
        
        total_flow.append(flow)
        node_index.append(node)
    
    node_index = [i for _, i in sorted(zip(total_flow, range(len(total_flow))), reverse=True)]
    

    
    two_thirds_biggest_flow_index = node_index[:int(len(node_index) * (1/3))]
    
    for individual in range(population):
        
        assign_array = [0] * n_node
        hub_array = [0] * n_node
        
        if individual < int(population*(1/3)):
        
            hubs = np.random.choice(two_thirds_biggest_flow_index, n_hub, replace=False)
        else:
            hubs = np.random.choice(n_node, n_hub, replace=False)
        
        for item in range(len(assign_array)):
            for hub in hubs:
                if item == hub:
                    assign_array[item] = hub

        for item in range(len(assign_array)):
            if item not in set(hubs):
                assign_array[item] = int(np.random.choice(hubs))

        for item in range(len(assign_array)):
            if assign_array[item] == item:
                hub_array[item] = 1
            else: hub_array[item] = 0
        
        assign_array_set.append(assign_array)
        hub_array_set.append(hub_array)
        
#     print(two_thirds_biggest_flow_index)
    
    return assign_array_set, hub_array_set

#### Random hub selection
note: unused for this case

In [ ]:
def random_initial_solution_GA(n_hub, n_node, population):
    
    assign_array_set = []
    hub_array_set = []
    
    for individual in range(population):
    
        assign_array = [0] * n_node
        hub_array = [0] * n_node
        hubs = np.random.choice(n_node, n_hub, replace=False)

        for item in range(len(assign_array)):
            for hub in hubs:
                if item == hub:
                    assign_array[item] = hub

        for item in range(len(assign_array)):
            if item not in set(hubs):
                assign_array[item] = int(np.random.choice(hubs))

        for item in range(len(assign_array)):
            if assign_array[item] == item:
                hub_array[item] = 1
            else: hub_array[item] = 0
        
        assign_array_set.append(assign_array)
        hub_array_set.append(hub_array)
    
    return assign_array_set, hub_array_set

### Fitness Calculation Evaluation

In [ ]:
def calculate_fitness(assign_array_set,flow_matrix,cost_matrix):
    fitness_values = []
    for individual in assign_array_set:
        fitness_values.append(total_cost(individual,flow_matrix,cost_matrix))
  
    return fitness_values
        
        

## Selection

Roulette Wheel

In [ ]:
def roulette_wheel_selection(assign_array, hub_array, fitness_values):
    # Calculate the selection probabilities
    fitness_sum = np.sum(fitness_values)
    selection_probabilities = fitness_values / fitness_sum
    
    # Calculate the cumulative probabilities
    cumulative_probabilities = np.cumsum(selection_probabilities)

    
    # Select two individuals from the population
    selected_indices = []
    while len(selected_indices) < 2:
        r = np.random.rand()
        for j in range(len(cumulative_probabilities)):
            if j in selected_indices:
                continue
            if r < cumulative_probabilities[j]:
                selected_indices.append(j)
                break
#     print(selected_indices)
    # Return the selected individuals
    parents_assign_array = [assign_array[i] for i in selected_indices]
    parents_hub_array = [hub_array[i] for i in selected_indices]
    return parents_assign_array, parents_hub_array


Tournament Selection (unused)

In [ ]:
import numpy as np

def tournament_selection(assign_array, hub_array, fitness_values, tournament_size):
    
    
    
    # Select individuals from the population based on tournament size
    
    fitness_values = np.array(fitness_values)
    selected_indices = [0,0]
    
    while all(x == selected_indices[0] for x in selected_indices):
        selected_indices = []
        for i in range(2):
            tournament_indices = np.random.choice(range(len(assign_array)), tournament_size, replace=False)
            tournament_fitness_values = []
            for index in tournament_indices:
                tournament_fitness_values.append(fitness_values[index])
                selected_index = tournament_indices[np.argmax(tournament_fitness_values)]
            selected_indices.append(selected_index)
    
    # Return the selected individuals
    parents_assign_array = [assign_array[i] for i in selected_indices]
    parents_hub_array = [hub_array[i] for i in selected_indices]
    return parents_assign_array, parents_hub_array


### Reproduction
Crossover with repairment

In [ ]:
def single_point_crossover_with_repairment(parent_assign, parent_hub, n_hub, flow_matrix, cost_matrix):
    
    offspring_hub = [[0], [0]]
    offspring_assign = [[0], [0]]

#     while np.sum(offspring_hub[0]) != n_hub and np.sum(offspring_hub[1]) != n_hub:
        # Choose a random crossover point
    crossover_point = np.random.randint(0, len(parent_assign[0]))

    # Generate the offspring
    offspring_hub[0] = parent_hub[0][:crossover_point] + parent_hub[1][crossover_point:]
    offspring_hub[1] = parent_hub[1][:crossover_point] + parent_hub[0][crossover_point:]
    offspring_assign[0] = parent_assign[0][:crossover_point] + parent_assign[1][crossover_point:]
    offspring_assign[1] = parent_assign[1][:crossover_point] + parent_assign[0][crossover_point:]



    for item in range(2):
        hub_indices = [i for i, x in enumerate(offspring_hub[item]) if x == 1]
#         print(hub_indices)

        if len(hub_indices) > n_hub:
            while len(hub_indices) > n_hub:
            
                random_index = random.randint(0, len(hub_indices)-1)  # generate random index
                hub_indices.pop(random_index)
        elif len(hub_indices) < n_hub:
            while len(hub_indices) < n_hub:
                all_hub = list(range(len(parent_assign[0])))
                possible_hub = [i for i in all_hub if i not in hub_indices]
                random_index = random.choice(possible_hub)
                hub_indices.append(random_index)
            
            
        
        offspring_hub[item] = [0] * len(parent_assign[0])                                       
                                          
        for index in hub_indices:
            offspring_hub[item][index] = 1
        
    arranged_offspring_assign = []

    # Adjustment function
    for each_offspring_assign, each_offspring_hub in zip(offspring_assign, offspring_hub):

        hub_indices = [i for i, x in enumerate(each_offspring_hub) if x == 1]

        for node in range(len(each_offspring_assign)):
            if each_offspring_hub[node] == 1:
                each_offspring_assign[node] = node
            if each_offspring_hub[node] == 0:
                if each_offspring_assign[node] in hub_indices:
                    continue
                min_cost_index = list(hub_indices)[np.argmin(cost_matrix[list(hub_indices), each_offspring_assign[node]])]
                each_offspring_assign[node] = min_cost_index
            
        arranged_offspring_assign.append(each_offspring_assign)                         
    
    return arranged_offspring_assign, offspring_hub

### Mutation

In [ ]:
def mutation_shift(arranged_offspring_assign, offspring_hub):
    
    offspring_assign = []
    
    for offspring in arranged_offspring_assign:
#         random_number = np.random.random()
        selected_index = None
#         if random_number < 0.5:
        hub_indices = set(offspring)
        selected_index = np.random.randint(0,len(offspring))
        while selected_index in hub_indices:
            selected_index = np.random.randint(0,9)

        hub_indices = list(hub_indices)
        hub_indices.remove(offspring[selected_index])
        offspring[selected_index] = np.random.choice(hub_indices)
        offspring_assign.append(offspring)

    return offspring_assign, offspring_hub

In [ ]:
def mutation_exchange(arranged_offspring_assign, offspring_hub):
    
    offspring_assign = []
    
    for offspring in arranged_offspring_assign:
#         random_number = np.random.random()
#         if random_number < 0.5:
        hub_indices = set(offspring)
        hub1, hub2  = np.random.choice(list(hub_indices), 2, replace=False)
        node1, node2 = np.random.choice(list((set(range(len(offspring)))) - hub_indices), 2, replace=False)

        node1 = np.random.choice([i for i, x in enumerate(offspring) if x == hub1])
        node2 = np.random.choice([i for i, x in enumerate(offspring) if x == hub2])

#             while offspring[node1] != hub1 and offspring[node2] != hub2:
#                 node1, node2 = np.random.choice(list((set(range(len(offspring))))-hub_indices),2, replace=False)

        offspring[node1] = hub2
        offspring[node2] = hub1
        
        offspring_assign.append(offspring)

        
    return offspring_assign, offspring_hub


### Populate Offspring

In [ ]:
def populate_offspring(population, assign_array, hub_array, n_hub, fitness_values, tournament_size,flow_matrix, cost_matrix):
    
    new_assign_array_set = []
    new_hub_array_set = []
    
    offsprings_assign = []
    offsprings_hub = []
    
    while len(new_assign_array_set) < population:
#         parents_assign, parents_hub = tournament_selection(assign_array, hub_array, fitness_values, tournament_size)
        parents_assign, parents_hub = roulette_wheel_selection(assign_array, hub_array, fitness_values)
        
        arranged_offspring_assign, offspring_hub = single_point_crossover_with_repairment(parents_assign, parents_hub, n_hub, flow_matrix, cost_matrix)
        random_number = np.random.random()
        if random_number < 0.5:
            offspring_assigns, offspring_hubs = mutation_shift(arranged_offspring_assign, offspring_hub)
        else: 
            offspring_assigns, offspring_hubs = mutation_exchange(arranged_offspring_assign, offspring_hub)
        for each_offspring_assign, each_offspring_hub in zip(offspring_assigns, offspring_hubs):
            if len(new_assign_array_set) < population:
                new_assign_array_set.append(each_offspring_assign)
                new_hub_array_set.append(each_offspring_hub)
    
#     for item1, item2 in new_assign_array_set:
#         offsprings_assign.append(item1)
#         offsprings_assign.append(item2)
    
#     for item1, item2 in new_hub_array_set:
#         offsprings_hub.append(item1)
#         offsprings_hub.append(item2)
             
    return new_assign_array_set, new_hub_array_set
#     return offsprings_assign, offsprings_hub
        

###  Replacement
Elitism with modification

In [ ]:
def combination_replacement(offsprings_assign, offsprings_hub, assign_array, hub_array, flow_matrix, cost_matrix, fitness_values ):
    
    
    
#     offsprings_assign = []
#     offsprings_hub = []
        
#     for item1, item2 in new_assign_array_set:
#         offsprings_assign.append(item1)
#         offsprings_assign.append(item2)
    
#     for item1, item2 in new_hub_array_set:
#         offsprings_hub.append(item1)
#         offsprings_hub.append(item2)

    
    fitness_value_offsprings = calculate_fitness(offsprings_assign,flow_matrix,cost_matrix)
    
    offsprings_fitness_df=pd.DataFrame({'offspring':offsprings_assign,'hub':offsprings_hub, 'fitness_value':fitness_value_offsprings})
    offsprings_fitness_df.sort_values(by='fitness_value', ascending=False,inplace=True)
    offsprings_fitness_df.reset_index(drop=True, inplace=True)
    
#     offsprings_assign=offsprings_fitness_df['offspring'].to_list()
#     offsprings_hub = offsprings_fitness_df['hub'].to_list()
        
    fitness_value_parents = fitness_values.copy()
    
#     print(len(fitness_values))
#     print(len(assign_array))
    
    parents_fitness_df=pd.DataFrame({'parent':assign_array,'hub':hub_array, 'fitness_value':fitness_value_parents})
    parents_fitness_df.sort_values(by='fitness_value', ascending=True,inplace=True)
    parents_fitness_df.reset_index(drop=True, inplace=True)
    
#     assign_array=parents_fitness_df['parents'].to_list()
#     hub_array = parents_fitness_df['hub'].to_list()
    

    
    new_generation_assign = []
    new_generation_hub = []
    
    for pairwise in range(len(offsprings_assign)):
        
        if offsprings_fitness_df['fitness_value'][pairwise] <= parents_fitness_df['fitness_value'][pairwise]:
            new_generation_assign.append(offsprings_fitness_df['offspring'][pairwise])
            new_generation_hub.append(offsprings_fitness_df['hub'][pairwise])
#             print('parent')
            
        else:
            new_generation_assign.append(parents_fitness_df['parent'][pairwise])
            new_generation_hub.append(parents_fitness_df['hub'][pairwise])
#             print('son')
    
    
    return  new_generation_assign, new_generation_hub
#     return offsprings_fitness_df

    
    

#### Generational replacement

In [ ]:
def generational_replacement(offsprings_assign, offsprings_hub):
    
    new_generation_assign =  offsprings_assign.copy()
    new_generation_hub = offsprings_hub.copy()
    
    return new_generation_assign, new_generation_hub

### Setting up the Tabu Search part

### Neighboring Structure 1

Change spoke to different hub

In [ ]:
def NS_1(become_tabu1, become_tabu2, best_solution, current_solution, temp_solution, hub_indices, flow_matrix, cost_matrix, best_cost, tabu_key):
    best_solution = None
    become_tabu1 = None
    become_tabu2 = None
    best_cost = float('inf')
    current_solution = current_solution.copy()
#     print('1')
#     counter = 0

    for node in range(len(temp_solution)):
        if node in hub_indices or node in tabu_key:
            continue
        for current_hub in hub_indices:
            for new_hub in hub_indices:
                if current_hub == new_hub:
                    continue
                
                if temp_solution[node] == current_hub:
                    temp_solution[node] = new_hub
                
                    temp_cost = total_cost(temp_solution, flow_matrix, cost_matrix)
#                     print(temp_solution)
    #                 print(temp_cost)
                    if temp_cost < best_cost:
                        best_solution = temp_solution.copy()
                        best_cost = temp_cost
                        become_tabu1 = node
                    temp_solution = current_solution.copy()
#                     counter +=1
                    
#     print(counter)            
    return best_solution, best_cost, become_tabu1, become_tabu2
        

### Neighbor Solution 2
Swapping two hubs

In [ ]:
def NS_2(become_tabu1, become_tabu2, best_solution, current_solution, temp_solution, hub_indices, flow_matrix, cost_matrix, best_cost, tabu_key):
    
#     print('2')  
    best_solution = None
    become_tabu1 = None
    become_tabu2 = None
    best_cost = float('inf')
    current_solution = current_solution.copy()
#     counter = 0
    
    
  
    for hub1 in hub_indices:
        for hub2 in hub_indices:
            if hub1 == hub2:
                continue
            if hub1 in tabu_key or hub2 in tabu_key:
                continue
            
#             print(hub1, hub2)
            for node in range(len(temp_solution)):
                if node == hub1 or node == hub2:
                    continue
                if temp_solution[node] == hub1:
                    temp_solution[node] = hub2
                elif temp_solution[node] == hub2:
                    temp_solution[node] = hub1
                    
                if temp_solution == current_solution:
                    continue
                
#             print(temp_solution)
            temp_cost = total_cost(temp_solution, flow_matrix, cost_matrix)
                

            if temp_cost < best_cost:
                best_solution = temp_solution.copy()
                best_cost = temp_cost
                become_tabu1 = hub1
                become_tabu2 = hub2

            temp_solution = current_solution.copy()
#             counter += 1
#     print(tabu_key)
#     print(best_solution)
#     print(counter)
    return best_solution, best_cost, become_tabu1, become_tabu2

### Neighboring Structure 3
Swapping spoke to hub, and hub to spoke

In [ ]:
def NS_3(become_tabu1, become_tabu2, best_solution, current_solution, temp_solution, hub_indices, flow_matrix, cost_matrix, best_cost, tabu_key):
    
#     print('3')
    best_solution = None
    become_tabu1 = None
    become_tabu2 = None
    best_cost = float('inf')
    current_solution = current_solution.copy()
#     counter = 0
    
    for node in range(len(temp_solution)): 
        if node in hub_indices or node in tabu_key:
                continue
        
        for hub in hub_indices:
            
            if hub in tabu_key:
                continue
                
            temp_solution[node] = node
            for other_node in range(len(temp_solution)):
                if other_node == node:
                    continue
                if temp_solution[other_node] == hub:
                    temp_solution[other_node] = node
                    
            temp_solution[hub] = node        
            temp_cost = total_cost(temp_solution, flow_matrix, cost_matrix)
#             print('node '+str(node))
#             print('hub '+str(hub))
#             print(temp_solution)

            if temp_cost < best_cost:
                best_solution = temp_solution.copy()
                best_cost = temp_cost
                become_tabu1 = node
                become_tabu2 = hub
            temp_solution = current_solution.copy()
#             counter += 1
#     print(counter)
    return best_solution, best_cost, become_tabu1, become_tabu2

#### Neighboring Structure 4
Swapping spoke to hub, and hub to spoke, only for spoke-hub that are interconnected

In [ ]:
def NS_4(become_tabu1, become_tabu2, best_solution, current_solution, temp_solution, hub_indices, flow_matrix, cost_matrix, best_cost, tabu_key):
    
#     print('4')
    best_solution = None
    become_tabu1 = None
    become_tabu2 = None
    best_cost = float('inf')
    
    current_solution = current_solution.copy()
#     counter = 0
    
    for node in range(len(temp_solution)): 
        if node in hub_indices or node in tabu_key:
                continue
        
        for hub in hub_indices:
                        
            if temp_solution[node] != hub or hub in tabu_key:
                continue
            
            temp_solution[node] = node
            for other_node in range(len(temp_solution)):
                if other_node == node:
                    continue
                if temp_solution[other_node] == hub:
                    temp_solution[other_node] = node
                    
            temp_solution[hub] = node        
            temp_cost = total_cost(temp_solution, flow_matrix, cost_matrix)
#             print('node '+str(node))
#             print('hub '+str(hub))
            
#             print(temp_solution)
            if temp_cost < best_cost:
                best_solution = temp_solution.copy()
                best_cost = temp_cost
                become_tabu1 = node
                become_tabu2 = hub
            temp_solution = current_solution.copy()
#             counter += 1
#     print(counter)
    return best_solution, best_cost, become_tabu1, become_tabu2

In [ ]:
NS_set = [NS_1, NS_2, NS_3, NS_4]



### Main Tabu Search Algorithm Model

In [ ]:
def tabu_search(current_solution,current_cost, flow_matrix, cost_matrix, n_hub, n_node, tabu_tenure, tabu_list):

    
    hub_indices = set(current_solution)

    temp_solution = current_solution.copy()

 

    if bool(tabu_list):
        keys_to_delete = []
        for key, value in tabu_list.items():
            tabu_list[key] = value - 1
            if tabu_list[key] <= 0:
                keys_to_delete.append(key)
        for key in keys_to_delete:
            del tabu_list[key]

    tabu_key = list(tabu_list.keys())


    best_solution = None
    become_tabu1 = None
    become_tabu2 = None
    best_cost = float('inf')

    while best_solution == None:
        best_solution, best_cost, become_tabu1, become_tabu2 = np.random.choice(NS_set)(become_tabu1, become_tabu2, best_solution, current_solution, temp_solution, hub_indices, flow_matrix, cost_matrix, best_cost, tabu_key)


    tabu_list[become_tabu1] = tabu_tenure
    if become_tabu2 != None:
        tabu_list[become_tabu2] = tabu_tenure


    
#     return final_best_solution, final_best_cost, best_iteration, updated, current_time - start_time
    return best_solution, best_cost

### Main Model: Genetic Algorithm with Tabu Seach integrated within it

In [ ]:
import time

def genetic_algorithm(n_node, n_hub, population, cost_matrix, flow_matrix, initial_solution, tabu_tenure, termination_time):
    
    final_best_cost = float('inf')
    final_best_solution = None
    
    best_solution_list = []
    final_best_solution_list = []
    final_best_cost_list = []
    time_stamp = []
    tabu_list = {}
    tabu_key = [] 
    
    if initial_solution == 'flowbased':
        assign_array, hub_array = weighted_flowbased_initial_solution_GA(flow_matrix, n_hub, n_node, population)
    elif initial_solution == 'costbased':
        assign_array, hub_array = weighted_costbased_initial_solution_GA(cost_matrix, n_hub, n_node, population)
    else:
        assign_array, hub_array = random_initial_solution_GA(n_hub, n_node, population)
    
    updated = 0
    start_time = time.time()
    current_time = start_time
    
    while current_time - start_time < termination_time:
        
        # calculate fitness value
        fitness_values = calculate_fitness(assign_array, flow_matrix, cost_matrix)
        
        best_cost = float('inf')
        best_solution = None
        current_cost = None
        
        for index, cost in enumerate(fitness_values):
            if cost < best_cost:
                best_cost = cost
                best_solution = assign_array[index]
                enter_tabu_search = index
                
        
        
        TS_solution, TS_cost = tabu_search(best_solution,best_cost, flow_matrix, cost_matrix, n_hub, n_node, tabu_tenure, tabu_list)
        
        if TS_cost < best_cost:
            assign_array[index] = TS_solution
            best_cost = TS_cost
            best_solution = TS_solution
        
          
        # create set of new generation population through selection and reproduction
        new_assign_array_set, new_hub_array_set = populate_offspring(population, assign_array, hub_array, n_hub, fitness_values, 10, flow_matrix, cost_matrix)
        
        # apply replacement strategy to get new generation
        new_generation_assign, new_generation_hub = combination_replacement(new_assign_array_set, new_hub_array_set, assign_array, hub_array, flow_matrix, cost_matrix, fitness_values)
        
        
        best_solution_list.append(best_solution)
        
        if best_cost < final_best_cost:
            final_best_cost = best_cost
            final_best_solution = best_solution.copy()
            best_iteration = len(best_solution_list) - 1
            updated += 1
            
        final_best_solution_list.append(final_best_solution)
        final_best_cost_list.append(final_best_cost)
        
        assign_array = new_generation_assign.copy()
        hub_array = new_generation_hub.copy()
        
        current_time = time.time()
        time_stamp.append(current_time - start_time)
    
    

    fig, ax = plt.subplots()
    ax.plot(range(len(final_best_solution_list)), [total_cost(best_solution, flow_matrix, cost_matrix) for best_solution in final_best_solution_list])
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Total Cost (Final Best Solution)')
    ax.set_title('Final Best Solution')
    
    plt.show()

    return final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp


## CAB25 Hub 3

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(200 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = genetic_algorithm(25, 3, 50, cost_matrix_25, flow_matrix_25, initial_solution = 'flowbased', tabu_tenure = 8, termination_time = 6)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('HYBRID_GATS_CAB25_3Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='GA_RGP100_7')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
test = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
test

## CAB25 Hub 5

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = genetic_algorithm(25, 5, 50, cost_matrix_25, flow_matrix_25, initial_solution = 'flowbased', tabu_tenure = 8, termination_time = 10)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('HYBRID_GATS_CAB25_5Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='GA_RGP100_7')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
test = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
test

## TR55 Hub 3

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = genetic_algorithm(55, 3, 50, cost_matrix_55, flow_matrix_55, initial_solution = 'flowbased', tabu_tenure = 15, termination_time = 35)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('HYBRID_GATS_TR55_3Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='GA_RGP100_7')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
test = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
test

## TR55, Hub 5

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = genetic_algorithm(55, 5, 50, cost_matrix_55, flow_matrix_55, initial_solution = 'flowbased', tabu_tenure = 15, termination_time = 50)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('HYBRID_GATS_TR55_5Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='GA_RGP100_7')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
test = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
test

## TR81, Hub 5

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = genetic_algorithm(81, 5, 50, cost_matrix_81, flow_matrix_81, initial_solution = 'flowbased', tabu_tenure = 25, termination_time = 120)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('HYBRID_GATS_TR81_5Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='GA_RGP100_7')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
test = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
test

## TR81, Hub 7

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = genetic_algorithm(81, 7, 50, cost_matrix_81, flow_matrix_81, initial_solution = 'flowbased', tabu_tenure = 25, termination_time = 140)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('HYBRID_GATS_TR81_7Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='GA_RGP100_7')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
test = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
test

## RGP100 Hub 7

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = genetic_algorithm(100, 7, 50, cost_matrix_100, flow_matrix_100, initial_solution = 'flowbased', tabu_tenure = 35, termination_time = 400)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('HYBRID_GATS_RGP100_7Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='GA_RGP100_7')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
test = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
test

## RGP100 Hub 10

In [ ]:
import time


import pandas as pd
result_df = pd.DataFrame(columns=['final_best_solution', 'final_best_cost', 'best_iteration', 'execution_time','best_cost_list', 'time_stamp'])

for i in range(10):
    random.seed(100 + i)
    start_time = time.time()
    final_best_solution, final_best_cost, best_iteration, updated, final_best_cost_list, time_stamp = genetic_algorithm(100, 10, 50, cost_matrix_100, flow_matrix_100, initial_solution = 'flowbased', tabu_tenure = 35, termination_time = 500)
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Execution time:", end_time - start_time, "seconds")
    print(final_best_cost)
    print(final_best_solution)
    print(best_iteration)

    result_df.loc[i] = [final_best_solution, final_best_cost, best_iteration, execution_time, final_best_cost_list, time_stamp]


with pd.ExcelWriter('HYBRID_GATS_RGP100_10Hub.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='GA_RGP100_7')
    
print(result_df.loc[:, 'final_best_cost'].mean())
print(result_df.loc[:, 'execution_time'].mean())
test = result_df.iloc[:,:4].sort_values(by='final_best_cost', ascending = True)
test